In [1]:
from deps.dobot_api import DobotApiDashboard, DobotApiMove
from deps import utils
from deps import cv_core
import numpy as np
import matplotlib.pyplot as plt
import cv2
from pynput import keyboard
import time
import ipyparallel as ipp

In [6]:
rc = ipp.Cluster(n=1).start_and_connect_sync()
e0 = rc[0]
e0.block = False
e0.activate('0')

Starting 1 engines with <class 'ipyparallel.cluster.launcher.LocalEngineSetLauncher'>
100%|██████████| 1/1 [00:05<00:00,  5.52s/engine]


In [ ]:
def sort_anchors(inp):
    maxx = sorted(np.squeeze(inp), key=lambda x: x[0])

    left = maxx[:2]
    right = maxx[-2:]

    l_sorty = sorted(left, key=lambda x: x[1])
    r_sorty = sorted(right, key=lambda x: x[1])

    return np.vstack((l_sorty, r_sorty))

In [15]:
dash = DobotApiDashboard('192.168.1.6', 29999)
move = DobotApiMove('192.168.1.6', 30003)
keys = utils.Keyboard(dash)

In [16]:
dash.ClearError()
dash.EnableRobot()

'0,{},EnableRobot();'

In [ ]:
utils.report_mode(dash)
utils.report_error(dash)

In [ ]:
dash.DisableRobot()

In [ ]:
utils.get_pose(dash)

In [ ]:
keys = utils.Keyboard(dash)
keys.execute()

In [14]:
manmove = utils.ManualMove(move, dash)
manmove.execute()

Special key pressed: Key.left
Special key pressed: Key.left
Special key pressed: Key.left
Special key pressed: Key.left
Special key pressed: Key.left
Special key pressed: Key.left
Special key pressed: Key.left
Special key pressed: Key.left
Special key pressed: Key.left
Special key pressed: Key.left
Special key pressed: Key.left
Special key pressed: Key.left
Special key pressed: Key.left
Special key pressed: Key.left
Special key pressed: Key.left
Special key pressed: Key.left
Special key pressed: Key.left
Special key pressed: Key.left
Special key pressed: Key.left
Special key pressed: Key.left
Special key pressed: Key.left
Special key pressed: Key.left
Special key pressed: Key.left
Special key pressed: Key.left
Special key pressed: Key.left
Special key pressed: Key.left
Special key pressed: Key.left
Special key pressed: Key.left
Special key pressed: Key.left
Special key pressed: Key.left
Special key pressed: Key.left
Special key pressed: Key.left
Special key pressed: Key.left
Special ke

In [18]:
w = 1600
h = 1200
xys = [(arr[0], arr[1]) for arr in coords]
robot_coor = utils.assign_corners(xys, reverse=True)
pix_coor = utils.assign_corners([(w/4, h/4), (3*w/4, h/4), (w/4, 3*h/4), (3*w/4, 3*h/4)])

In [8]:
robot_coor

{'ul': (250.834315, -7.259071),
 'ur': (256.516215, 45.54948),
 'lr': (297.274674, 46.244652),
 'll': (292.467159, -7.257414)}

In [19]:
features_mm_to_pixels_dict = {}
for key, value in robot_coor.items():
    features_mm_to_pixels_dict[value] = pix_coor[key]


tf_mtx = cv_core.compute_tf_mtx(features_mm_to_pixels_dict)

In [ ]:
print(np.array(robot_coor['ul']) - np.array(robot_coor['ur']))
print(np.array(robot_coor['ul']) - np.array(robot_coor['ll']))
print(np.array(robot_coor['ur']) - np.array(robot_coor['lr']))
print(np.array(robot_coor['lr']) - np.array(robot_coor['ll']))

## Get the anchors coordinates:

In [ ]:
cap = cv2.VideoCapture(0)
cap = cv_core.set_res(cap, (1600, 1200))
cv2.namedWindow('frame',  cv2.WINDOW_NORMAL)
cv2.resizeWindow('frame', 1348, 1011)

cameraMatrix = np.load('./cam_matrices/cam_mtx.npy')
dist = np.load('./cam_matrices/dist.npy')
newCameraMatrix = np.load('./cam_matrices/newcam_mtx.npy')

all_anchors = []

idx = 1
while True:
    ret, frame = cap.read()
    frame = cv2.undistort(frame, cameraMatrix, dist, None, newCameraMatrix)
    
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    detected_circles = cv2.HoughCircles(image=gray,
                                        method=cv2.HOUGH_GRADIENT,
                                        dp=1,
                                        minDist=30,
                                        param1=100,
                                        param2=40,
                                        minRadius=20,
                                        maxRadius=40
                                        )


    if detected_circles is not None:
        plot_img = frame.copy()
        circles = np.uint16(np.around(detected_circles))
        if np.squeeze(circles).shape[0] == 4:
            all_anchors.append(sort_anchors(circles))

        for i in circles[0,:]:
            center = (i[0], i[1])
            cv2.circle(plot_img, center, 1, (255, 0 ,0), 3)
            radius = i[2]
            cv2.circle(plot_img, center, radius, (0, 255 ,0), 3)

        cv2.putText(plot_img, f"Frame {idx}", (25,25), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0,255,0), 2)

        # for i in all_anchors[0]:
        #     center = (int(i[0]), int(i[1]))
        #     cv2.circle(plot_img, center, 1, (0, 0 ,255), 3)
        #     radius = i[2]
        #     cv2.circle(plot_img, center, int(radius), (0, 255 ,0), 3)
    idx += 1
    cv2.imshow('frame', plot_img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()

plot_img = frame.copy()
means = np.mean(all_anchors, axis = 0)
for i in means:
    center = (int(i[0]), int(i[1]))
    cv2.circle(plot_img, center, 1, (0, 0 ,255), 3)
    radius = i[2]
    cv2.circle(plot_img, center, int(radius), (0, 0, 255), 3)
cv2.putText(plot_img, "Captured:", (25,25), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0,255,0), 2)
cv2.imshow('frame', plot_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
# print("Detecting Anchors")
# while True:
#     cap = cv2.VideoCapture(0)
#     cap = cv_core.set_res(cap, (1600, 1200))
#     centers = cont.wait_for_anchors(cap, show = True)
#     cap.release()
#     if len(centers) == 4:
#         break

centers = means[:, :-1]
xys = [(arr[0], arr[1]) for arr in keys.coords]
pix_coor = utils.assign_corners(centers)
robot_coor = utils.assign_corners(xys, reverse=True)


features_mm_to_pixels_dict = {}
for key, value in robot_coor.items():
    features_mm_to_pixels_dict[value] = pix_coor[key]


tf_mtx = cv_core.compute_tf_mtx(features_mm_to_pixels_dict)

In [10]:
features_mm_to_pixels_dict

{(250.834315, -7.259071): (400.0, 300.0),
 (256.516215, 45.54948): (1200.0, 300.0),
 (297.274674, 46.244652): (1200.0, 900.0),
 (292.467159, -7.257414): (400.0, 900.0)}

In [45]:
utils.get_pose(dash)

X = 338.326466
Y = 38.617479
Z = -73.43631
r = -1.208932


array([338.326466,  38.617479, -73.43631 ,  -1.208932])

In [12]:
cont = cv_core.Contours()

def on_change(val): pass

offset = 40
cap = cv2.VideoCapture(0)
cap = cv_core.set_res(cap, cv_core.camera_res_dict['1200'])

cameraMatrix = np.load('./cam_matrices/cam_mtx.npy')
dist = np.load('./cam_matrices/dist.npy')
newCameraMatrix = np.load('./cam_matrices/newcam_mtx.npy')

cv2.namedWindow('frame',  cv2.WINDOW_NORMAL)
cv2.resizeWindow('frame', 1348, 1011)
cv2.createTrackbar('Manual Lock', 'frame', 0, 1, on_change)
cv2.createTrackbar('Mask Offset', 'frame', offset, 150, on_change)
cv2.setMouseCallback('frame', cont.mousecallback)

idx = 0
prev_point = (0,0,0)
while(True):
    ret, frame = cap.read()
    #frame = cv2.undistort(frame, cameraMatrix, dist, None, newCameraMatrix)
    val = cv2.getTrackbarPos('Manual Lock', 'frame')
    offset = cv2.getTrackbarPos('Mask Offset', 'frame')
    if val == 1:
        cont.locked = True
    else:
        cont.locked = False

    if cont.best_circ is None:
        while cont.best_circ is None:
            pt = cont.find_contours(frame, 10, offset)
    else:
        pt = cont.find_contours(frame, 10, offset)
    print(pt)
    print(type(pt))
    if pt is None:
        pt = (800, 600, 300)
        
    a,b,r = pt
    plot_img = frame.copy()
    cv2.circle(plot_img, (a, b), r, (3, 162, 255), 2)
    cv2.circle(plot_img, (a, b), 1, (0, 0, 255), 3)
    cv2.circle(plot_img, (a, b), r - offset, (0, 255, 0), 2)
    cv2.circle(plot_img, cont.big_circ[:2], cont.big_circ[2], (0, 0, 255), 2)
    cv2.putText(plot_img, f"{cont.big_circ[2]*2}px = 60mm", (cont.big_circ[0]-25, cont.big_circ[1] - cont.big_circ[2] - 10),
        cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0, 0, 255), 2)
    
    cv2.drawContours(plot_img, cont.singular, -1,(0,255, 0),2)
    cv2.drawContours(plot_img, cont.clusters, -1,(0,0,255),2)

    if cont.selected:
        cv2.drawContours(plot_img, cont.selected, -1,(255,0,0),2)

    for c in cont.singular:
        # compute the center of the contour
        M = cv2.moments(c)
        cX = int(M["m10"] / M["m00"])
        cY = int(M["m01"] / M["m00"])
        # draw the contour and center of the shape on the image
        #cv2.drawContours(plot_img, [c], -1, (0, 255, 0), 2)
        cv2.circle(plot_img, (cX, cY), 2, (0, 0, 255), -1)
        # cv2.putText(plot_img, f"{cX},{cY}", (cX - 20, cY - 20),
        # cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        cv2.putText(plot_img, f"{cv2.contourArea(c)}", (cX - 20, cY - 20),
        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 1)
    #out.write(with_contours)

    for c in cont.clusters:
        # compute the center of the contour
        M = cv2.moments(c)
        cX = int(M["m10"] / M["m00"])
        cY = int(M["m01"] / M["m00"])
        # draw the contour and center of the shape on the image
        #cv2.drawContours(plot_img, [c], -1, (0, 255, 0), 2)
        cv2.circle(plot_img, (cX, cY), 2, (0, 0, 255), -1)
        # cv2.putText(plot_img, f"{cX},{cY}", (cX - 20, cY - 20),
        # cv2.FONT_HERSHEY_SIMPLEX, 0.35, (0, 255, 0), 1)

        # cv2.putText(plot_img, f"{cv2.contourArea(c)}", (cX - 20, cY - 20),
        # cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 1)


    if prev_point is pt:
        idx += 1
    else:
        idx = 0
    prev_point = pt


    if idx >= 30:
        message = 'LOCKED'
        color = (0,255,0)
    else:
        message = 'SEARCHING'
        color = (0,0,255)

    cv2.putText(plot_img, "TARGET:", (25,25), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0,255,0), 2)
    cv2.putText(plot_img, f"{message}", (125,25), cv2.FONT_HERSHEY_SIMPLEX, 0.75, color, 2)
    cv2.putText(plot_img, f"Found: {len(cont.singular)}", (25,50), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0,255,0), 2)

    #cv2.setWindowProperty('frame', cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)
    cv2.imshow('frame',plot_img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

[ WARN:0@2910.763] global /opt/conda/conda-bld/opencv-suite_1656606573658/work/modules/videoio/src/cap_gstreamer.cpp (862) isPipelinePlaying OpenCV | GStreamer warning: GStreamer: pipeline have not been created


[790 572 444]
<class 'numpy.ndarray'>
[790 572 444]
<class 'numpy.ndarray'>
[790 572 444]
<class 'numpy.ndarray'>
[790 572 444]
<class 'numpy.ndarray'>
[790 572 444]
<class 'numpy.ndarray'>
[783 574 438]
<class 'numpy.ndarray'>
[783 574 438]
<class 'numpy.ndarray'>
[783 574 438]
<class 'numpy.ndarray'>
[783 574 438]
<class 'numpy.ndarray'>
[783 574 438]
<class 'numpy.ndarray'>
[783 574 438]
<class 'numpy.ndarray'>
[783 574 438]
<class 'numpy.ndarray'>
[783 574 438]
<class 'numpy.ndarray'>
[783 574 438]
<class 'numpy.ndarray'>
[783 574 438]
<class 'numpy.ndarray'>
[783 574 438]
<class 'numpy.ndarray'>
[783 574 438]
<class 'numpy.ndarray'>
[783 574 438]
<class 'numpy.ndarray'>
[783 574 438]
<class 'numpy.ndarray'>
[783 574 438]
<class 'numpy.ndarray'>
[783 574 438]
<class 'numpy.ndarray'>
[783 574 438]
<class 'numpy.ndarray'>
[783 574 438]
<class 'numpy.ndarray'>
[783 574 438]
<class 'numpy.ndarray'>
[796 569 394]
<class 'numpy.ndarray'>
[796 569 394]
<class 'numpy.ndarray'>
[796 569 394

In [72]:
dash.DisableRobot()

'0,{},DisableRobot();'

In [14]:
dash.ClearError()
dash.EnableRobot()

NameError: name 'dash' is not defined

In [70]:
move.MoveJog('Z+')

'0,{},MoveJog(Z+);'

In [71]:
dash.ResetRobot()

'0,{},ResetRobot();'

In [20]:
#centers_cuboids = cont.contour_centers(cont.singular)
centers_cuboids = cont.contour_centers(cont.selected)

dash.ClearError()
dash.EnableRobot()

#utils.default_pos(move)
for center in centers_cuboids:
    X, Y, _ = tf_mtx @ (center[0], center[1], 1)
    move.MovL(X+3.16, Y+0.32, -97.593536,  0)

    # move.MovL(X, Y, -91,  0)
    # time.sleep(0.5)
    break
    move.MovL(X, Y, -70,  0)

#utils.default_pos(move)

In [58]:
manmove = utils.ManualMove(move, dash)
manmove.execute()

Special key pressed: Key.down
Special key pressed: Key.down
Special key pressed: Key.right
Special key pressed: Key.left
Special key pressed: Key.right
Special key pressed: Key.left
Special key pressed: Key.left
Special key pressed: Key.right
Special key pressed: Key.esc
wow


In [59]:
utils.get_pose(dash)

X = 306.646639
Y = 5.535293
Z = -76.166901
r = 0.0


array([306.646639,   5.535293, -76.166901,   0.      ])

In [57]:
utils.get_pose(dash)

X = 303.478346
Y = 5.212162
Z = -76.168182
r = 0.0


array([303.478346,   5.212162, -76.168182,   0.      ])

In [60]:
306.646639 - 303.478346

3.1682930000000056

In [61]:
5.535293 - 5.212162

0.32313100000000006

In [ ]:
keys = utils.Keyboard(dash)
keys.execute()
xys = [(arr[0], arr[1]) for arr in keys.coords]

In [ ]:
robot_coor = utils.assign_corners(xys, reverse=True)

In [ ]:
mean_ux = np.mean([robot_coor['ul'][0], robot_coor['ur'][0]])
mean_lx = np.mean([robot_coor['ll'][0], robot_coor['lr'][0]])

mean_uy = np.mean([robot_coor['ur'][1], robot_coor['lr'][1]])
mean_ly = np.mean([robot_coor['ll'][1], robot_coor['ul'][1]])

In [ ]:
well_plate_xs = np.linspace(mean_ux, mean_lx, 24)
well_plate_ys = np.linspace(mean_uy, mean_ly, 16)

In [ ]:
utils.report_error(dash)

In [ ]:
dash.ClearError()
dash.EnableRobot()

utils.default_pos(move)
time.sleep(0.5)
for x in well_plate_xs:
    for y in well_plate_ys:
        move.MovL(x, y, -70,  0)
        time.sleep(0.5)
        move.MovL(x, y, -75,  0)
        time.sleep(0.5)
        move.MovL(x, y, -70,  0)
        time.sleep(0.5)

utils.default_pos(move)

In [ ]:
utils.default_pos(move)

In [ ]:
well_plate_coords = np.concatenate(np.stack(np.meshgrid(well_plate_xs, well_plate_ys), axis = 2))

In [ ]:
dash.ClearError()
dash.EnableRobot()
centers_cuboids = cont.contour_centers(cont.singular)

utils.default_pos(move)
for idx, center in enumerate(centers_cuboids):
    X, Y, _ = tf_mtx @ (center[0], center[1], 1)
    move.MovL(X, Y, -70,  0)
    time.sleep(0.5)
    move.MovL(X, Y, -90,  0)
    time.sleep(0.5)
    move.MovL(X, Y, -70,  0)
    time.sleep(0.5)
    
    x,y = well_plate_coords[idx]
    move.MovL(x, y, -70,  0)
    time.sleep(0.5)
    move.MovL(x, y, -80,  0)
    time.sleep(0.5)
    move.MovL(x, y, -70,  0)
    time.sleep(0.5)

utils.default_pos(move)

In [8]:
%%px0
from deps import utils
from deps.dobot_api import DobotApiDashboard, DobotApiMove
dash = DobotApiDashboard('192.168.1.6', 29999)
move = DobotApiMove('192.168.1.6', 30003)
manmove = utils.ManualMove(move, dash)
manmove.execute()
coords = manmove.coords

<AsyncResult(%px): pending>

In [10]:
ar = e0.pull('coords')
coords = ar.get()

In [11]:
coords

[array([241.580408,  -7.801041, -97.593536, -17.605   ]),
 array([283.247129,  -7.798385, -97.55735 , -17.604952]),
 array([288.427727,  44.163045, -97.551666, -17.604927]),
 array([247.754079,  43.223319, -97.523239, -17.604847])]

In [22]:
def draw_cross(w,h,frame):
    w = int(w)
    h = int(h)
    frame[h-5:h+6, w-1:w+2] = [0, 255, 0]
    frame[h-1:h+2, w-5:w+6] = [0, 255, 0]
    return frame

cameraMatrix = np.load('./cam_matrices/cam_mtx.npy')
dist = np.load('./cam_matrices/dist.npy')
newCameraMatrix = np.load('./cam_matrices/newcam_mtx.npy')

cap = cv2.VideoCapture(0)
cap = cv_core.set_res(cap, cv_core.camera_res_dict['1200'])
cv2.namedWindow('frame',  cv2.WINDOW_NORMAL)
cv2.resizeWindow('frame', 1348, 1011)

coords = []

while(True):
    ret, frame = cap.read()
    h,w = frame.shape[:-1]
    frame = cv2.undistort(frame, cameraMatrix, dist, None, newCameraMatrix)
    # gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)s

    # dst = cv2.cornerHarris(gray, blockSize=2, ksize=3, k=0.04)
    # frame[dst > 0.01 * dst.max()] = [0, 255, 0]
    frame = draw_cross(w/2, h/2, frame)
    frame = draw_cross(w/4, h/4, frame)
    frame = draw_cross(3*w/4, h/4, frame)
    frame = draw_cross(w/4, 3*h/4, frame)
    frame = draw_cross(3*w/4, 3*h/4, frame)
    cv2.imshow('frame',frame)

    k = cv2.waitKey(1)
    if k == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

[ WARN:0@6155.187] global /opt/conda/conda-bld/opencv-suite_1656606573658/work/modules/videoio/src/cap_gstreamer.cpp (862) isPipelinePlaying OpenCV | GStreamer warning: GStreamer: pipeline have not been created


In [53]:
dash.ClearError()
dash.EnableRobot()

'0,{},EnableRobot();'

In [54]:
X, Y, _ = tf_mtx @ (999,434, 1)
move.MovL(X, Y, -76.168182,  0)

'0,{},MovL(303.122401,23.493025,-76.168182,0.000000);'

In [ ]:
utils.default_pos(move)

In [ ]:
frame.shape